In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/W499510_deep_learning/Project/"
#!pip install yahoo_finance_api2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/W499510_deep_learning/Project


In [0]:
!pip install yahoo_finance_api2

In [0]:
import pandas as pd
import numpy as np
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
from datetime import datetime


def dataloader(stock_name,
              start = '1995-01-01 01:00:00',
              end = '2018-12-31 04:00:00',verbose = True):

  my_share = share.Share(stock_name)
  symbol_data = None

  try:
      symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,30,
                                            share.FREQUENCY_TYPE_DAY,1)
      if verbose:
        print(stock_name+" has been downloaded")
      
  except YahooFinanceError as e:
      print(e.message)
      sys.exit(1)
  df = pd.DataFrame(symbol_data)
  
  df['timestamp'] = [datetime.fromtimestamp(t/1000) for t in df['timestamp']]

  # test = df[(df['timestamp'] >= '2016-1-1 01:00:00') & (df['timestamp'] <= '2018-12-31 04:00:00')]
  return df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]

In [0]:
dat_comp_train = pd.read_csv("./data/train.csv")
stock_lst = [s.strip("\n") for s in list(dat_comp_train.keys() )]
stock_lst.pop(0)
stock_lst.pop()
res = {}
for i,stock_name in enumerate(stock_lst,0):
    # if i> 2: break
    try:
        df = dataloader(stock_name,start = '2018-05-31 01:00:00', end = '2019-01-09 04:00:00',verbose = False).copy()
        print(stock_name,i+1, len(df))
        if len(df) == 153:
            res[str(stock_name)] = list(df['close'])
    except:
        print(stock_name, i+1, "Failed")

'Cash'

In [0]:
import pandas as pd
dat = pd.read_csv("./data/test.csv")
res = ( (dat - dat.shift(periods= 1 , fill_value= 0))/dat )

The  1<sup>st</sup> hack: The following code select the top 50 stocks and last 50 stocks of the day to buy or short. We distribute money over the top 50 stocks and sell short over 50 negative return stocks.

In [0]:
target = pd.read_csv("./data/sample_submission.csv")
target['Predicted'] = 0

for i, row in res.iterrows():
    if i >2:break
    row_lst = list(row)
    name_lst = list(row.keys() )
    print(min(row_lst))
    inds = sorted(range(len(row_lst)), key=lambda i: row_lst[i], reverse=True)[:50]
    inds_neg = sorted(range(len(row_lst)), key=lambda i: row_lst[i], reverse=True)[:50]
    S = 0
    for j,ind in enumerate(inds,1):
        id = str(i-1)+"_"+ name_lst[ind]
        target.loc[target["Id"]==id,"Predicted"] = row[name_lst[ind]]
        S += row[name_lst[ind]]
    for j,ind in enumerate(inds,1):
        id = str(i-1)+"_"+ name_lst[ind]
        target.loc[target["Id"]==id,"Predicted"] = sum(target.loc[target["Id"]==id,"Predicted"])/S
        print("{}th Winner for day {} is {}, the rate of return is {:.2f}%, share is {:.2f}% ".format(j,i-1,
                                                                                              name_lst[ind],
                                                                                              100*row[name_lst[ind]],
                                                                                              100*sum(target.loc[target["Id"]==id,"Predicted"])/S))

1.0
1th Winner for day -1 is A , the rate of return is 100.00%, share is 0.00% 
2th Winner for day -1 is AAL , the rate of return is 100.00%, share is 0.00% 
3th Winner for day -1 is AAP , the rate of return is 100.00%, share is 0.00% 
4th Winner for day -1 is AAPL , the rate of return is 100.00%, share is 0.00% 
5th Winner for day -1 is ABBV , the rate of return is 100.00%, share is 0.00% 
6th Winner for day -1 is ABC , the rate of return is 100.00%, share is 0.00% 
7th Winner for day -1 is ABMD , the rate of return is 100.00%, share is 0.00% 
8th Winner for day -1 is ABT , the rate of return is 100.00%, share is 0.00% 
9th Winner for day -1 is ACN , the rate of return is 100.00%, share is 0.00% 
10th Winner for day -1 is ADBE , the rate of return is 100.00%, share is 0.00% 
11th Winner for day -1 is ADI , the rate of return is 100.00%, share is 0.00% 
12th Winner for day -1 is ADM , the rate of return is 100.00%, share is 0.00% 
13th Winner for day -1 is ADP , the rate of return is 1

The 2<sup>nd</sup> hack: distribute money over all the positive stock on that day and sell short on every stock

In [0]:
import numpy as np
target = pd.read_csv("./data/sample_submission.csv")
target.drop(columns=['Predicted'], inplace= True)

id_lst,pred_lst = [], []

for i, row in res.iterrows():
    if i == 0: continue
    row_lst = list(row)
    name_lst = list(row.keys() )

    S_pos = sum(r for r in row_lst if r>0)
    S_neg = sum(r for r in row_lst if r<0)

    id_lst += [str(i-1)+'_'+name for name in name_lst]
    pred_lst += [r/S_pos if r >0 else r/abs(S_neg) for r in row_lst]
    # pred_lst += [r/S_pos if r >0 else 0 for r in row_lst]
target2 = pd.DataFrame({'Id':id_lst, 'Predicted':pred_lst})
target = target.merge(target2, on = "Id",how = 'left')
target.fillna(0, inplace= True)
target['Id'] = target['Id'].apply(lambda id: id.split(" ")[0])


In [0]:
target.to_csv("submission_2.csv",index = False)